# WebApp Prototyping with Google PaLM

In [20]:
OUTPUT_PATH = "output/palm/java"

In [2]:
technology_stack = {
    "database": "PostgreSQL",
    "backend": {
        "language": "Java",
        "framework": "Spring Framework 6.1.0",
        "testing_framework": "JUnit5"
    },
    "frontend": {
        "language": "Javascript",
        "framework": "Vue"
    }
}

In [25]:
def output_response(data: str, filename: str):
    output_file_path = OUTPUT_PATH + "/" + filename
    with open(output_file_path, 'w') as f:
        f.write(data)

## Setup

In [3]:
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

In [4]:
google_api_key = os.getenv('GOOGLEAI_API_KEY')

In [5]:
palm.configure(api_key=google_api_key)

In [6]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='Text Bison',
      description='Model targeted for text generation.',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [7]:
from google.api_core import retry
@retry.Retry()
def get_completion(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

In [8]:
responses = {}

## A technical specification of the business domain based on a client request

In [9]:
brand_name = "On all fours"

In [10]:
business_requirements = f"""
As an entrepreneur stepping into the online retail space, 
I am lookinng for an expertise in developing an e-commerce website for my upcoming business venture.
I aim to launch a dynamic, user-friendly platform dedicated to selling shoes. 
Below, I outline my requirements and expectations for this project.

Business Overview:
Our brand, {brand_name}, is poised to enter the online shoe market. 
We aim to offer a wide range of footwear for various demographics, 
focusing on quality, style, and comfort. 
Our objective is to create a seamless and engaging online shopping 
experience for our customers.

Project Scope:

Website Design and Development:

Aesthetic that resonates with our brand's image.
Responsive design compatible with desktop, tablet, and mobile devices.
User-friendly navigation and intuitive layout.
E-Commerce Functionality:

Product catalog with high-quality images and detailed descriptions.
Shopping cart and secure checkout process.
Payment gateway integration (e.g., PayPal, credit/debit cards).
Inventory management system.
Customer Account Features:

User registration and login.
Order history and tracking.
Wishlist and product review system.
SEO and Analytics:

SEO optimization for better visibility in search engines.
Integration with analytics tools to track website performance and customer behavior.
Security:

SSL certification for secure transactions.
Compliance with data protection regulations.
Maintenance and Support:

Ongoing website maintenance post-launch.
Technical support and troubleshooting.
Timeline and Budget:

We are aiming to launch the website within [specify timeline, e.g., 6 months].
Our budget for this project is in the range of [specify budget].
Proposal Requirements:
Please provide a detailed proposal that includes:

Your approach to website design and development.
An overview of the technology stack you plan to use.
A breakdown of costs and time estimates for each phase of the project.
Examples of similar e-commerce websites you have developed.
Information about post-launch support and maintenance services.
"""

## Project specification

In [34]:
prompt = f"""
You are a an experienced web application architect.                      \\
Your team prefers using the following technology stack:                  \\ 
- for database: {technology_stack["database"]}                           \\
- for backend:                                                           \\  
    - language: {technology_stack["backend"]["language"]}                \\ 
    - framework: {technology_stack["backend"]["framework"]}              \\
    - testing framework: {technology_stack["backend"]["testing_framework"]}    \\
- for frontend:                                                          \\
    - language: {technology_stack["frontend"]["language"]}               \\
    - framework: {technology_stack["frontend"]["framework"]}             \\

Your task is to provide a technical description of a project             \\
using a client's request.                                                \\

Create a technical specification based on your team preferences and       \\
the client requirements provided below and delimited by triple backticks. \\

Business requirements: ```{business_requirements}```
"""
completion = get_completion(prompt)
responses["project_specification"] = completion.result

In [35]:
output_file_name = "project_specification.md"
data = responses["project_specification"]
output_response(data, output_file_name)

## The data model specification

In [13]:
prompt = f"""
You are a database architect. \\
Your task is to create a technical specification of the data model \\
using overall project specification.

Technical specification of the data model should include: \\
- list of the entities with data type definitions and comments
- list of the attributes with data type definitions and comments
- description of dependecies between the entities with comments

Write a data model specification based on the \\
project specification delimited by triple backticks.

Business requirements: ```{responses["project_specification"]}```
"""
completion = get_completion(prompt)
responses["data_model_specification"] = completion.result

In [30]:
output_file_name = "data_model_specification.md"
data = responses["data_model_specification"]
output_response(data, output_file_name)

## Database DDL script

In [16]:
prompt = f"""

You are a database architect.                                        \\
Your task is to create a database *DDL script*                         \\
using this *data model specification*                                    \\
delimited by the triple backticks ```{responses["data_model_specification"]}```. \\                                                                   \\
- Use snake case style for the tables' and fields' names.              \\
- To each table and to each field of the table provide extensive explanation in form of a comment \\
                                                                     \\
Use this *project specification*                                     \\
delimited by double quotes: "{responses["project_specification"]}"   \\
to choose a particular SQL dialect.                                  \\
"""
completion = get_completion(prompt)
responses["ddl_script"] = completion.result

In [31]:
output_file_name = "ddl_script.sql"
data = responses["ddl_script"]
output_response(data, output_file_name)

## Backend code outline

In [18]:
prompt = f"""
You are an experienced backend developer. \\
Your task is to implement backend module for a WEB application \\
based on                                    \\
- this *project specification* delimited by double quotes: "{responses["project_specification"]}" \\
and                                           \\
- the database *DDL script* delimited by triple backticks: ```{responses["ddl_script"]}```.     \\

Use *project specification* to choose a programming language and framework.
============================

Use *DDL script* as follows:
============================
For each table definition in the *DDL script* provide \\
- a repository class with methods defining CRUD operations \\
- an entity class with properties and getters and setters for each of the properties \\
- a controller class, enabling interactions with the frontend \\
For each method provide a comment explaining the logic. \\
"""

completion = get_completion(prompt)
responses["backend_code"] = completion.result

In [32]:
output_file_name = "backend_code.md"
data = responses["backend_code"]
output_response(data, output_file_name)